In [2]:
import geopandas as gpd
import pyogrio
from data_pipeline.etl.sources.census.etl import CensusETL
from data_pipeline.etl.sources.tribal.etl import TribalETL

import time

begin = time.time()

In [3]:
# Load Tribal geojson
tribal_gdf = gpd.read_file(
    TribalETL().NATIONAL_TRIBAL_GEOJSON_PATH,
    # Use `pyogrio` because it's vectorized and faster.
    engine="pyogrio",
)

tribal_gdf

,tribalId,landAreaName,Classification,geometry
0,LAR0001,Cheyenne River LAR,1,"MULTIPOLYGON (((-100.49935 45.47125, -100.4993..."
1,LAR0002,Crow Creek LAR,1,"POLYGON ((-99.42137 44.27733, -99.42138 44.273..."
2,LAR0003,Flandreau LAR,1,"MULTIPOLYGON (((-96.56655 44.08786, -96.57165 ..."
3,LAR0004,Fort Berthold LAR,1,"POLYGON ((-102.78362 47.99900, -102.78192 47.9..."
4,LAR0005,Lake Traverse (Sisseton) LAR,1,"MULTIPOLYGON (((-97.28946 45.76084, -97.28955 ..."
...,...,...,...,...
592,{0886416F-643E-497E-89D3-E9CC0240158D},Chilkat,None,POINT (-135.88440 59.40390)
593,{2029C35B-86D7-4751-A946-EA0772C81A80},Chilkoot,None,POINT (-135.44500 59.23580)
594,{24DF6536-95CB-4964-94DF-16E440ABCA92},Craig,None,POINT (-133.14830 55.47640)
595,{ACDE097A-9BDA-4FCA-9DB7-297DA6B73F88},Douglas,None,POINT (-134.41970 58.30190)


In [4]:
# Drop the points from the Tribal data (because these cannot be joined to a (Multi)Polygon tract data frame)
tribal_gdf = tribal_gdf[tribal_gdf.geom_type != "Point"]
tribal_gdf

,tribalId,landAreaName,Classification,geometry
0,LAR0001,Cheyenne River LAR,1,"MULTIPOLYGON (((-100.49935 45.47125, -100.4993..."
1,LAR0002,Crow Creek LAR,1,"POLYGON ((-99.42137 44.27733, -99.42138 44.273..."
2,LAR0003,Flandreau LAR,1,"MULTIPOLYGON (((-96.56655 44.08786, -96.57165 ..."
3,LAR0004,Fort Berthold LAR,1,"POLYGON ((-102.78362 47.99900, -102.78192 47.9..."
4,LAR0005,Lake Traverse (Sisseton) LAR,1,"MULTIPOLYGON (((-97.28946 45.76084, -97.28955 ..."
...,...,...,...,...
365,TSA0354,Seminole TSA,None,"POLYGON ((-96.49048 34.90423, -96.49146 34.903..."
366,TSA0355,Seneca Cayuga TSA,None,"POLYGON ((-94.61803 36.62531, -94.62083 36.625..."
367,TSA0356,Tonkawa TSA,None,"POLYGON ((-97.24698 36.68082, -97.24697 36.677..."
368,TSA0357,Wichita Caddo and Delaware TSA,None,"POLYGON ((-97.99931 35.36425, -97.99948 35.360..."


In [5]:
# Load Census tracts geojson
census_tract_gdf = gpd.read_file(
    CensusETL.NATIONAL_TRACT_JSON_PATH,
    # Use `pyogrio` because it's vectorized and faster.
    engine="pyogrio",
)

census_tract_gdf

,STATEFP10,COUNTYFP10,TRACTCE10,GEOID10,NAME10,NAMELSAD10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,geometry
0,20,071,958100,20071958100,9581,Census Tract 9581,G5020,S,2016176814,0,+38.4804076,-101.8059837,"POLYGON ((-101.79971 38.69806, -101.79097 38.6..."
1,20,175,965600,20175965600,9656,Census Tract 9656,G5020,S,1603575701,2204351,+37.1805849,-100.8547406,"POLYGON ((-101.06766 37.20440, -101.06768 37.2..."
2,20,175,965700,20175965700,9657,Census Tract 9657,G5020,S,9466451,358282,+37.0625361,-100.9131437,"POLYGON ((-100.94250 37.06497, -100.94251 37.0..."
3,20,043,020300,20043020300,203,Census Tract 203,G5020,S,211593206,7045771,+39.7881238,-094.9734666,"POLYGON ((-94.95518 39.90129, -94.95475 39.901..."
4,20,043,020200,20043020200,202,Census Tract 202,G5020,S,352687026,2968059,+39.7540484,-095.1060098,"POLYGON ((-95.02575 39.88295, -95.02585 39.883..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
74129,35,049,000600,35049000600,6,Census Tract 6,G5020,S,1629471,0,+35.6758519,-105.9446097,"POLYGON ((-105.95207 35.67367, -105.95215 35.6..."
74130,35,049,000700,35049000700,7,Census Tract 7,G5020,S,1285597,0,+35.6802004,-105.9558818,"POLYGON ((-105.96221 35.67223, -105.96245 35.6..."
74131,35,049,000800,35049000800,8,Census Tract 8,G5020,S,1916797,0,+35.6805095,-105.9703558,"POLYGON ((-105.98159 35.67739, -105.98143 35.6..."
74132,35,049,000900,35049000900,9,Census Tract 9,G5020,S,2545563,0,+35.6692966,-105.9755351,"POLYGON ((-105.96362 35.67616, -105.96365 35.6..."


In [6]:
# Create a measure for the entire census tract area
census_tract_gdf["area_tract"] = census_tract_gdf.area
census_tract_gdf

/var/folders/lx/xmq8p65j71v9xq2bhsd2j5w40000gp/T/ipykernel_768/2956500515.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  census_tract_gdf["area_tract"] = census_tract_gdf.area


,STATEFP10,COUNTYFP10,TRACTCE10,GEOID10,NAME10,NAMELSAD10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,geometry,area_tract
0,20,071,958100,20071958100,9581,Census Tract 9581,G5020,S,2016176814,0,+38.4804076,-101.8059837,"POLYGON ((-101.79971 38.69806, -101.79097 38.6...",0.208156
1,20,175,965600,20175965600,9656,Census Tract 9656,G5020,S,1603575701,2204351,+37.1805849,-100.8547406,"POLYGON ((-101.06766 37.20440, -101.06768 37.2...",0.162976
2,20,175,965700,20175965700,9657,Census Tract 9657,G5020,S,9466451,358282,+37.0625361,-100.9131437,"POLYGON ((-100.94250 37.06497, -100.94251 37.0...",0.000995
3,20,043,020300,20043020300,203,Census Tract 203,G5020,S,211593206,7045771,+39.7881238,-094.9734666,"POLYGON ((-94.95518 39.90129, -94.95475 39.901...",0.022990
4,20,043,020200,20043020200,202,Census Tract 202,G5020,S,352687026,2968059,+39.7540484,-095.1060098,"POLYGON ((-95.02575 39.88295, -95.02585 39.883...",0.037373
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74129,35,049,000600,35049000600,6,Census Tract 6,G5020,S,1629471,0,+35.6758519,-105.9446097,"POLYGON ((-105.95207 35.67367, -105.95215 35.6...",0.000162
74130,35,049,000700,35049000700,7,Census Tract 7,G5020,S,1285597,0,+35.6802004,-105.9558818,"POLYGON ((-105.96221 35.67223, -105.96245 35.6...",0.000128
74131,35,049,000800,35049000800,8,Census Tract 8,G5020,S,1916797,0,+35.6805095,-105.9703558,"POLYGON ((-105.98159 35.67739, -105.98143 35.6...",0.000191
74132,35,049,000900,35049000900,9,Census Tract 9,G5020,S,2545563,0,+35.6692966,-105.9755351,"POLYGON ((-105.96362 35.67616, -105.96365 35.6...",0.000253


In [7]:
# Performing overlay funcion
gdf_joined = gpd.overlay(census_tract_gdf, tribal_gdf, how="union")

/var/folders/lx/xmq8p65j71v9xq2bhsd2j5w40000gp/T/ipykernel_768/1353983773.py:2: UserWarning: `keep_geom_type=True` in overlay resulted in 1123 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  gdf_joined = gpd.overlay(census_tract_gdf, tribal_gdf, how="union")


In [8]:
# Calculate overlap
# Calculating the areas of the newly-created geometries
gdf_joined["area_joined"] = gdf_joined.area

# Calculating the areas of the newly-created geometries in relation
# to the original grid cells
gdf_joined["tribal_area_as_a_share_of_tract_area"] = (
    gdf_joined["area_joined"] / gdf_joined["area_tract"]
)
gdf_joined

/var/folders/lx/xmq8p65j71v9xq2bhsd2j5w40000gp/T/ipykernel_768/2727120487.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_joined['area_joined'] = gdf_joined.area


,STATEFP10,COUNTYFP10,TRACTCE10,GEOID10,NAME10,NAMELSAD10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,area_tract,tribalId,landAreaName,Classification,geometry,area_joined,tribal_area_as_a_share_of_tract_area
0,20,043,020100,20043020100,201,Census Tract 201,G5020,S,454634616.0,2601186.0,+39.8206800,-095.2567279,0.048098,LAR0210,Iowa LAR,1,"POLYGON ((-95.33994 39.97506, -95.33994 39.975...",4.998139e-04,0.010391
1,20,013,480600,20013480600,4806,Census Tract 4806,G5020,S,882293538.0,1376818.0,+39.8596443,-095.6255187,0.093019,LAR0210,Iowa LAR,1,"POLYGON ((-95.45656 40.00025, -95.45528 40.000...",3.209294e-03,0.034502
2,31,147,964500,31147964500,9645,Census Tract 9645,G5020,S,677848509.0,6076731.0,+40.1522236,-095.5858870,0.072289,LAR0210,Iowa LAR,1,"MULTIPOLYGON (((-95.38162 40.02744, -95.38119 ...",1.476624e-03,0.020427
3,29,087,960300,29087960300,9603,Census Tract 9603,G5020,S,412869716.0,6745159.0,+39.9730230,-095.1479701,0.044239,LAR0210,Iowa LAR,1,"POLYGON ((-95.38119 40.02755, -95.38162 40.027...",1.965514e-07,0.000004
4,20,085,082600,20085082600,826,Census Tract 826,G5020,S,690868809.0,947758.0,+39.4553966,-095.6731404,0.072404,LAR0211,Kickapoo (Kansas) LAR,1,"POLYGON ((-95.71031 39.65308, -95.69902 39.653...",5.285627e-06,0.000073
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76317,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TSA0353,Sac and Fox TSA,None,"MULTIPOLYGON (((-96.62002 35.75143, -96.62001 ...",6.560647e-17,NaN
76318,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TSA0354,Seminole TSA,None,"MULTIPOLYGON (((-96.77536 35.03300, -96.77536 ...",7.207055e-18,NaN
76319,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TSA0355,Seneca Cayuga TSA,None,"POLYGON ((-94.61836 36.74340, -94.61836 36.743...",7.016721e-18,NaN
76320,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TSA0356,Tonkawa TSA,None,"MULTIPOLYGON (((-97.24698 36.69942, -97.24692 ...",2.612218e-17,NaN


In [9]:
# Aggregating the results
results = gdf_joined.groupby(["GEOID10", "landAreaName"]).agg(
    {"tribal_area_as_a_share_of_tract_area": "sum"}
)

results = results.reset_index()

results.to_csv(
    "~/Downloads/tribal_area_as_a_share_of_tract_area.csv", index=False
)

# Printing results
print(results)

          GEOID10        landAreaName  tribal_area_as_a_share_of_tract_area
0     01051030800    Poarch Creek LAR                              0.002467
1     01053970400    Poarch Creek LAR                              0.002367
2     01053970500    Poarch Creek LAR                              0.000682
3     01101005408    Poarch Creek LAR                              0.001391
4     02130000100  Annette Island LAR                              0.000038
...           ...                 ...                                   ...
2585  56013940300      Wind River LAR                              0.204039
2586  56013940400      Wind River LAR                              0.053289
2587  56017967900      Wind River LAR                              0.191189
2588  56033000600            Crow LAR                              0.000565
2589  56035000102      Wind River LAR                              0.000140

[2590 rows x 3 columns]


In [10]:
end = time.time()

print("Time taken to execute the ETL is", end - begin)

Time taken to execute the function is 140.10310292243958
